In [19]:
import sys
sys.path.append("../")

import csv
import pandas as pd
import numpy as np
import json
from itertools import repeat, chain
import utils

## Running to Collect ACS Data using Census API

In [29]:
# Lets set up our requests session. You can use teh creaet session utility or just pass the requests object
session = utils.create_session()

## State codes for all states & Washington DC
## As listed here - https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html
states = [i for i in range(1,57) if i not in [3, 7, 14, 43, 52]]
#states = [5,  25, 27, 33, 50]

In [30]:
## These codes and endpoints are from viewing the Census API documentation once you identify the correct table
## Here's a google doc of the codes we want to use

codes = {'acs/acs5':[ "B01002_001E", "B01002_002E", "B01002_003E", "B15003_001E", "B15003_017E","B15003_018E",
                    "B15003_019E", "B15003_020E", "B15003_021E", "B15003_022E", "B15003_023E", "B15003_024E", 
                    "B15003_025E", "B28002_001E", "B28002_002E", "B28002_003E", "B28002_004E", "B28002_005E", 
                    "B28002_006E", "B28002_007E", "B28002_008E", "B28002_009E", "B28002_010E", "B28002_011E", 
                     "B28002_012E", "B28002_013E", "B28003_001E", "B28003_002E", "B28003_003E", "B28003_004E",
                    "B28003_005E", "B28003_006E"],
         
        'acs/acs5/subject' : ["S2301_C03_001E", "S1701_C02_001E", "S1701_C01_001E", "S1901_C01_012E",
                             "S1101_C01_001E", "S1101_C01_002E", "S1101_C01_004E"],
         
        'acs/acs5/profile' : ['DP04_0142PE', 'DP04_0141PE', 'DP05_0001E', "DP04_0089E", "DP05_0077PE", "DP05_0071PE",
                             "DP05_0078PE", "DP05_0079PE", "DP05_0080PE", "DP05_0081PE", "DP05_0082PE", "DP05_0083PE"
                             ]}

acs_endpoints = ['acs/acs5', 'acs/acs5/subject', 'acs/acs5/profile'] 

# codes = {'acs/acs5':[ "B03002_001E","B03002_002E"]}


# acs_endpoints = ['acs/acs5'] 


#dfs = utils.collect_results(acs_endpoints, codes, states, session)
dfs = []
outputs = utils.gather_results(session, acs_endpoints, states, codes, dfs)

https://api.census.gov/data/2019/acs/acs5?get=NAME,B01002_001E,B01002_002E,B01002_003E,B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B28002_001E,B28002_002E,B28002_003E,B28002_004E,B28002_005E,B28002_006E,B28002_007E,B28002_008E,B28002_009E,B28002_010E,B28002_011E,B28002_012E,B28002_013E,B28003_001E,B28003_002E,B28003_003E,B28003_004E,B28003_005E,B28003_006E&for=tract:*&in=state:05&key=daeac6d558149025218d49696ca5c9988533bd37
https://api.census.gov/data/2019/acs/acs5/subject?get=NAME,S2301_C03_001E,S1701_C02_001E,S1701_C01_001E,S1901_C01_012E,S1101_C01_001E,S1101_C01_002E,S1101_C01_004E&for=tract:*&in=state:05&key=daeac6d558149025218d49696ca5c9988533bd37
https://api.census.gov/data/2019/acs/acs5/profile?get=NAME,DP04_0142PE,DP04_0141PE,DP05_0001E,DP04_0089E,DP05_0077PE,DP05_0071PE,DP05_0078PE,DP05_0079PE,DP05_0080PE,DP05_0081PE,DP05_0082PE,DP05_0083PE&for=tract:*&in=state:05&key=daeac6d558149025218d49696ca5c99885

In [34]:
outputs.columns

Index(['NAME', 'B01002_001E', 'B01002_002E', 'B01002_003E', 'B15003_001E',
       'B15003_017E', 'B15003_018E', 'B15003_019E', 'B15003_020E',
       'B15003_021E', 'B15003_022E', 'B15003_023E', 'B15003_024E',
       'B15003_025E', 'B28002_001E', 'B28002_002E', 'B28002_003E',
       'B28002_004E', 'B28002_005E', 'B28002_006E', 'B28002_007E',
       'B28002_008E', 'B28002_009E', 'B28002_010E', 'B28002_011E',
       'B28002_012E', 'B28002_013E', 'B28003_001E', 'B28003_002E',
       'B28003_003E', 'B28003_004E', 'B28003_005E', 'B28003_006E', 'state',
       'county', 'tract', 'S2301_C03_001E', 'S1701_C02_001E', 'S1701_C01_001E',
       'S1901_C01_012E', 'S1101_C01_001E', 'S1101_C01_002E', 'S1101_C01_004E',
       'DP04_0142PE', 'DP04_0141PE', 'DP05_0001E', 'DP04_0089E', 'DP05_0077PE',
       'DP05_0071PE', 'DP05_0078PE', 'DP05_0079PE', 'DP05_0080PE',
       'DP05_0081PE', 'DP05_0082PE', 'DP05_0083PE'],
      dtype='object')

In [36]:
np.unique(outputs.state).shape #Should be 51 for 50 states and DC

(51,)

In [37]:
outputs.to_csv("../data/census_data.csv")